# Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 

**Основное**
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

### Решение

In [845]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [846]:
K = 5 # Число рекомендаций
N_POPULAR = 500 # Топ популярных для префильтра
N_CANDIDATES = 100 # Число кандидатов

#### Загрузка данных и схема валидации

In [847]:
data_train = pd.read_csv('../raw_data/retail_train.csv')
item_features = pd.read_csv('../raw_data/product.csv')
user_features = pd.read_csv('../raw_data/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

#### Схема валидации

In [848]:
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data_train[data_train['week_no'] < data_train['week_no'].max() - 
                              (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_train_lvl_2 = data_train[(data_train['week_no'] >= data_train['week_no'].max() - 
                               (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data_train['week_no'] < data_train['week_no'].max() - (val_lvl_2_size_weeks))]
data_val_lvl_2 = data_train[data_train['week_no'] >= data_train['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


#### Первичная фильтрация данных

In [849]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, take_n_popular=N_POPULAR) #item_features=item_features

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 501


#### Обучение модели первого уровня

In [850]:
recommender = MainRecommender(data_train_lvl_1, als=False, own=True)

#### Получаем список актуальных покупок и кандидатов для data_train_lvl_2

In [851]:
def get_actual(data):
    # Получение актуальных покупок
    result = data.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    return result

In [852]:
def get_candidates(model, data):
    # Получение кандидатов на покупку
    data['candidates'] = data['user_id'].apply(lambda x: model.get_own_recommendations(x, N=N_CANDIDATES))
    return data

In [853]:
result_lvl_2 = get_actual(data_train_lvl_2)
result_lvl_2 = get_candidates(recommender, result_lvl_2)
result_lvl_2.head(2)

,user_id,actual,candidates
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[940947, 9527290, 986947, 995242, 861272, 1006..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1075368, 8090521, 1040807, 940947, 5569230, 1..."


#### Размечаем данные для второй модели

In [854]:
def get_X_data(data):
    # Формирование X данных
    s = data.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'item_id'
    return data.drop(['actual', 'candidates'], axis=1).join(s)

In [855]:
def mark_target(data, result):
    # Разметка данных
    users_lvl_2 = get_X_data(result)
    targets_lvl_2 = data[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1  # тут только покупки 
    targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    return targets_lvl_2

In [856]:
X_y = mark_target(data_train_lvl_2, result_lvl_2)
X_y.head(2)

,user_id,item_id,target
0,1,940947,1.0
1,1,940947,1.0


#### Новые фичи

In [857]:
def featuring(data_target, data_train=None):
    # Добавляем внешние данные
    data_target = data_target.merge(item_features, on='item_id', how='left')
    data_target = data_target.merge(user_features, on='user_id', how='left')
    
    if data_train is not featuring.__defaults__[0]:
        # Средняя цена товара
        featuring.item_mean_price = pd.DataFrame()
        featuring.item_mean_price[['item_id', 'sum_sales']] = \
            data_train.groupby(['item_id'])['sales_value'].sum().reset_index()[['item_id', 'sales_value']]
        featuring.item_mean_price['sum_quantity'] = data_train.groupby(['item_id'])['quantity'].sum().reset_index()['quantity']
        featuring.item_mean_price['item_mean_price'] = \
            featuring.item_mean_price['sum_sales'] / featuring.item_mean_price['sum_quantity']
        featuring.item_mean_price = featuring.item_mean_price.drop(['sum_sales', 'sum_quantity'], axis=1)

        # Среднее число продаж товара
        featuring.item_mean_quantity = pd.DataFrame()
        featuring.item_mean_quantity[['item_id', 'item_mean_quantity']] = \
            data_train.groupby(['item_id'])['quantity'].mean().reset_index()[['item_id', 'quantity']]

        # Средняя сумма покупок пользователем в категории 
        featuring.user_department_mean_sales = pd.DataFrame()
        featuring.user_department_mean_sales[['user_id', 'department', 'user_department_mean_sales']] = \
            data_train.merge(item_features, on='item_id', how='left').groupby(['user_id', 'department'])\
            ['sales_value'].mean().reset_index()

        # Средняя сумма покупок пользователем товара
        featuring.user_item_mean_sales = pd.DataFrame()
        featuring.user_item_mean_sales[['user_id', 'item_id', 'user_item_mean_sales']] = \
            data_train.groupby(['user_id', 'item_id'])['sales_value'].mean().reset_index()

    # Добавляем новые фичи в датасет
    data_target = data_target.merge(featuring.item_mean_price, on='item_id', how='left')
    data_target = data_target.merge(featuring.item_mean_quantity, on='item_id', how='left')
    data_target = data_target.merge(featuring.user_department_mean_sales, on=['user_id', 'department'], how='left')
    data_target = data_target.merge(featuring.user_item_mean_sales, on=['user_id', 'item_id'], how='left')
    
    data_target[['item_mean_price', 
                 'item_mean_quantity', 
                 'user_department_mean_sales', 
                 'user_item_mean_sales']] = \
        data_target[['item_mean_price', 
                     'item_mean_quantity', 
                     'user_department_mean_sales', 
                     'user_item_mean_sales']].fillna(0)

    return data_target

In [858]:
X_y = featuring(X_y, data_train_lvl_2)

#### Кодируем категориальные признаки

In [859]:
# Все категориальные признаки
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc', 'sub_commodity_desc', 
             'curr_size_of_product', 'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc', 
             'household_size_desc', 'kid_category_desc']

# Категориальные признаки для трансформации
cat_feats_tr = ['department', 'brand', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product', 
    'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc', 
    'household_size_desc', 'kid_category_desc']

In [860]:
dd_le = defaultdict(LabelEncoder)

def fit_cat_encoder(data, cat_features):
    for c in cat_features:
        dd_le[c].fit(data[c].fillna("NA").append(pd.Series(['NA']))) 

def transform_cat_encoder(data, cat_features):
    for c in cat_features:
        data[c] = dd_le[c].transform(data[c].fillna("NA")) 
    return data

In [861]:
fit_cat_encoder(user_features, ['age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc', 
    'household_size_desc', 'kid_category_desc'])
fit_cat_encoder(item_features, ['department', 'brand', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product'])

#### Разбиваем на train и test и обучаем

In [862]:
def prepare_X(X):
    # Подготовка X данных перед обучением/предсказанием
    X = transform_cat_encoder(X, cat_feats_tr)
    X[cat_feats] = X[cat_feats].astype('category')
    return X

In [863]:
def split_train_test(data):
    # Разделение на X и y
    X_train = data.drop('target', axis=1)
    X_train = prepare_X(X_train)
    y_train = data['target']    
    return X_train, y_train

In [864]:
X_train, y_train = split_train_test(X_y)

In [865]:
# Обучаемые признаки
train_f = X_train.columns.drop(['user_id', 'item_id'])

In [866]:
lgb = RandomForestClassifier(
    n_estimators=10,
    verbose=False,
    n_jobs=8,
    random_state=27
)
lgb.fit(X_train[train_f], y_train)

RandomForestClassifier(n_estimators=10, n_jobs=8, random_state=27,
                       verbose=False)

#### Берем топ-k предсказаний для data_train_lvl_2, ранжированных по вероятности, для каждого юзера

In [867]:
def get_model2(model, X, k):
    # Получение данных модели второго уровня
    X['preds'] = model.predict_proba(X[train_f])[:, 1]
    result = X.sort_values(['user_id', 'preds'], ascending=False).groupby('user_id').head(k).\
        groupby('user_id')['item_id'].unique().reset_index()
    result.rename(columns={'item_id': 'model2'}, inplace=True)
    return result

In [868]:
model2 = get_model2(lgb, X_train, K)
model2.head(2)

,user_id,model2
0,1,"[940947, 995242]"
1,2,"[1133018, 901062, 861272, 1106523, 916122]"


#### Находим рекомендации по первой модели

In [869]:
def get_model1(model, data, k):
    # Получение данных модели первого уровня
    model1 = data.copy()
    model1['model1'] = model1['user_id'].apply(lambda x: model.get_own_recommendations(x, N=k))
    return model1[['user_id', 'model1']]

In [870]:
model1 = get_model1(recommender, result_lvl_2, K)
model1.head(2)

,user_id,model1
0,1,"[940947, 9527290, 986947, 995242, 861272]"
1,2,"[1075368, 8090521, 1040807, 940947, 5569230]"


#### Сливаем всё в одну таблицу

In [871]:
result_lvl_2 = result_lvl_2.merge(model1, on='user_id', how='left')
result_lvl_2 = result_lvl_2.merge(model2, on='user_id', how='left')
result_lvl_2.head(2)

,user_id,actual,candidates,model1,model2
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[940947, 9527290, 986947, 995242, 861272, 1006...","[940947, 9527290, 986947, 995242, 861272]","[940947, 995242]"
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1075368, 8090521, 1040807, 940947, 5569230, 1...","[1075368, 8090521, 1040807, 940947, 5569230]","[1133018, 901062, 861272, 1106523, 916122]"


#### Вычисляем метрики на data_train_lvl_2

In [872]:
def get_precision_at_k(data, field, k):
    # Вычисление усреднённого значения precision@k
    return data.apply(lambda x: precision_at_k(x[field], x['actual'], k), axis=1).mean()

In [873]:
print('precision@k для модели первого уровня (data_train_lvl_2):', get_precision_at_k(result_lvl_2, 'model1', K),
      '\nprecision@k для модели второго уровня (data_train_lvl_2):', get_precision_at_k(result_lvl_2, 'model2', K))

precision@k для модели первого уровня (data_train_lvl_2): 0.3424326833797586 
precision@k для модели второго уровня (data_train_lvl_2): 0.8139430516867843


### Получим расчет для data_val_lvl_2

In [874]:
# Валидация
data_train_1_2 = data_train[data_train['week_no'] < data_train['week_no'].max() - val_lvl_2_size_weeks]
data_val_lvl_2 = data_train[data_train['week_no'] >= data_train['week_no'].max() - val_lvl_2_size_weeks]
# Первичная фильтрация данных
data_train_1_2 = prefilter_items(data_train_1_2, take_n_popular=N_POPULAR)
# Обучение модели первого уровня
recommender = MainRecommender(data_train_1_2, als=False, own=True)
# Получаем список актуальных покупок и кандидатов для data_val_lvl_2
result_lvl_2 = get_actual(data_val_lvl_2)
result_lvl_2 = get_candidates(recommender, result_lvl_2)
# Формируем данные для второй модели
X = get_X_data(result_lvl_2)
# Фичи
X = featuring(X)
# Предобработка данных перед предсказанием модели 2
X = prepare_X(X)
# Находим рекомендации по второй модели
model2 = get_model2(lgb, X, K)
# Находим рекомендации по первой модели
model1 = get_model1(recommender, result_lvl_2, K)
# Сливаем всё в одну таблицу
result_lvl_2 = result_lvl_2.merge(model1, on='user_id', how='left')
result_lvl_2 = result_lvl_2.merge(model2, on='user_id', how='left')
result_lvl_2.head(2)

,user_id,actual,candidates,model1,model2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[940947, 9527290, 986947, 995242, 861272, 1006...","[940947, 9527290, 986947, 995242, 861272]","[940947, 995242, 840361, 1082212, 865456]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1053690, 1092026, 951590, 9527494, 910032, 11...","[1053690, 1092026, 951590, 9527494, 910032]","[1053690, 1092026, 951590, 9527494, 910032]"


#### Вычисляем метрики на data_val_lvl_2

In [875]:
print('precision@k для модели первого уровня (data_val_lvl_2):', get_precision_at_k(result_lvl_2, 'model1', K),
      '\nprecision@k для модели второго уровня (data_val_lvl_2):', get_precision_at_k(result_lvl_2, 'model2', K))

precision@k для модели первого уровня (data_val_lvl_2): 0.30852105778648387 
precision@k для модели второго уровня (data_val_lvl_2): 0.35406464250734576


### Получим расчет для data_test

А теперь делаем почти то же самое, только обучим первую модель на полном датасете data_train и cделаем предсказание кандидатов для data_test. Затем полученных кандидатов ранжируем второй моделью и получим предсказания для data_test.

In [877]:
data_test = pd.read_csv('../raw_data/retail_test.csv')

In [878]:
# Первичная фильтрация данных
data_train = prefilter_items(data_train, take_n_popular=N_POPULAR)
# Обучение модели первого уровня
recommender = MainRecommender(data_train, als=False, own=True)
# Получаем список актуальных покупок и кандидатов для data_test
result_lvl = get_actual(data_test)
result_lvl = get_candidates(recommender, result_lvl)
# Формируем данные для второй модели
X = get_X_data(result_lvl)
# Фичи
X = featuring(X)
# Предобработка данных перед предсказанием модели 2
X = prepare_X(X)
# Находим рекомендации по второй модели
model2 = get_model2(lgb, X, K)
# Находим рекомендации по первой модели
model1 = get_model1(recommender, result_lvl, K)
# Сливаем всё в одну таблицу
result_lvl = result_lvl.merge(model1, on='user_id', how='left')
result_lvl = result_lvl.merge(model2, on='user_id', how='left')
result_lvl.head(2)

,user_id,actual,candidates,model1,model2
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[940947, 9527290, 986947, 995242, 861272, 1062...","[940947, 9527290, 986947, 995242, 861272]","[940947, 995242, 1005186, 840361, 1082212]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1075368, 8090521, 1040807, 940947, 861272, 55...","[1075368, 8090521, 1040807, 940947, 861272]","[861272, 901062, 1133018, 1106523, 916122]"


#### Вычисляем метрики на data_test

In [879]:
print('precision@k для модели первого уровня (data_test):', get_precision_at_k(result_lvl, 'model1', K),
      '\nprecision@k для модели второго уровня (data_test):', get_precision_at_k(result_lvl, 'model2', K))

precision@k для модели первого уровня (data_test): 0.2884880636604774 
precision@k для модели второго уровня (data_test): 0.3190185676392573


### Результирующая метрика precision@5 с использованием 2-х моделей для тестовых данных равна 0.3190185676392573

#### Сохранение результатов

In [881]:
result_lvl[['user_id', 'model2']].to_csv('recommendations.csv', index=False)